###Main params

In [1]:
remote_path = 'http://pacha.datawheel.us/economia/nene/occupational_status/'

local_path = '../data/'

engine_path = 'postgresql://localhost:5432/datachile'

###Imports

In [2]:
from urllib import request
import zipfile
import shutil
import os.path

import json
import pandas as pd
from sqlalchemy import create_engine

###Open file function

In [3]:
def loadFile(file_name):
    remote_file = remote_path + file_name
    local_file = local_path + file_name

    if not os.path.isfile(local_file):
        with request.urlopen(remote_file) as remote_csv,open(local_file, 'wb') as local_csv:
            shutil.copyfileobj(remote_csv, local_csv)
    
    return pd.read_csv(local_file,delimiter=",")

###Load file

In [4]:
df = loadFile('occupational_status.csv')
list(df)

['ano_encuesta',
 'mes_encuesta',
 'region_id',
 'age',
 'age_range_id',
 'sex_id',
 'icse_id',
 'isco_id',
 'isced_id',
 'occupied_id',
 'general_economic_condition',
 'fact']

###Complete NaN

In [5]:
df['isco_id'] = df['isco_id'].fillna('0')
df['icse_id'] = df['icse_id'].fillna('0')
df['isced_id'] = df['isced_id'].fillna('0')

###Rename, to int & load

In [6]:
df = df.rename(columns={'ano_encuesta':'year','mes_encuesta':'month'});
df = df.astype({'year':'int','month':'int','region_id':'int','age':'int','age_range_id':'int','icse_id':'int','sex_id':'int','isced_id':'int','isco_id':'int','occupied_id':'int','icse_id':'int','general_economic_condition':'int'})

engine = create_engine(engine_path)
df.to_sql('fact_occupational_status_nene', engine, schema='economy', if_exists='replace', index=False)

###Indexes & FK

In [7]:
engine.execute("""
ALTER TABLE economy.fact_occupational_status_nene
  ADD COLUMN date_id INTEGER; 
""")

engine.execute("""
UPDATE economy.fact_occupational_status_nene
SET date_id = dim_date.id
FROM public.dim_date
WHERE dim_date.the_year = economy.fact_occupational_status_nene.year
      AND dim_date.month_of_year = economy.fact_occupational_status_nene.month
      AND dim_date.day_of_month = 1
""")

engine.execute("""
CREATE INDEX fact_occupational_status_nene_region_id 
ON economy.fact_occupational_status_nene (region_id)
""")